# STEM Cell Population Gillespie Algorithm Fixation Analysis

In this example notebook we present an example of how to forward simulate a population of STEM cells in the context of a fixed size tumor. We assume a constant number of cells in the population at all times -- only the counts of the different species of cells change in time:

- wild type (WT)
- with cell intrinsic mutations that increase fitness (A)
- with mutations that give evolutionary advantage based on environmental factors such as level of cytokines (B).

Fot the purposes of this notebook we consider the environmental conditions such that the B cells always have a selectional advantage over their wilde type counterpart.

In [1]:
# Load necessary libraries
import os
import numpy as np
import pandas as pd
from scipy.stats import gamma
import cmmlinflam as ci
import matplotlib
import plotly.graph_objects as go
from matplotlib import pyplot as plt

## Plot output of Gillespie for the different species of cells

In [2]:
from plotly.subplots import make_subplots

colours = ['blue', 'red', 'green', 'purple', 'orange', 'black', 'gray', 'pink']
species = ['WT', 'A', 'B']

## Compute mean time to fixation and probability of fixation of A cells when no B are present

## No selectional advatange + No mutation

### Start from 1/2 and 1/2 WT ws A

In [3]:
# Set initial population state WT - A - B
initial_population = [50, 50, 0]

# Set baseline growth rate
alpha = 0.5

# Set selectional advantages for mutated cells
s = 0
r = 0

# Set mutation rates
mu_A = 0
mu_B = 0

# Coalesce into paramater vector
parameters = initial_population
parameters.extend([alpha, s, r, mu_A, mu_B])

In [4]:
# Instantiate algorithm
algorithm = ci.StemGillespie()

# Select number of simulations
num_simulations = 1000

computation_time = np.empty(num_simulations, dtype=np.int)
fixed_state = np.empty((num_simulations), dtype=np.str)

for sim in range(num_simulations):
    computation_time[sim], fixed_state[sim] = algorithm.simulate_fixation(parameters)

mean_computation_time = np.mean(computation_time)
prob_fix = (fixed_state == 'A').sum()/num_simulations

print('Average time to illness: ', mean_computation_time)
print('Probability of fixation of cell type A: ', prob_fix)

Average time to illness:  6874.256
Probability of fixation of cell type A:  0.503


### Start from 1 A vs all other WT

In [5]:
# Set initial population state WT - A - B
initial_population = [99, 1, 0]

# Set baseline growth rate
alpha = 0.5

# Set selectional advantages for mutated cells
s = 0
r = 0

# Set mutation rates
mu_A = 0
mu_B = 0

# Coalesce into paramater vector
parameters = initial_population
parameters.extend([alpha, s, r, mu_A, mu_B])

In [6]:
# Instantiate algorithm
algorithm = ci.StemGillespie()

# Select number of simulations
num_simulations = 1000

computation_time = np.empty(num_simulations, dtype=np.int)
fixed_state = np.empty((num_simulations), dtype=np.str)

for sim in range(num_simulations):
    computation_time[sim], fixed_state[sim] = algorithm.simulate_fixation(parameters)

mean_computation_time = np.mean(computation_time)
prob_fix = (fixed_state == 'A').sum()/num_simulations

print('Average time to illness: ', mean_computation_time)
print('Probability of fixation of cell type A: ', prob_fix)

Average time to illness:  462.081
Probability of fixation of cell type A:  0.007


In [7]:
# Plot transition probabilities
sep_algo = ci.StemGillespie()

sep_algo.N = int(np.sum(np.asarray(initial_population)))

sep_algo.alpha_A = alpha + s
sep_algo.alpha_B = alpha + r
sep_algo.alpha_WT = alpha

sep_algo.mu_A = mu_A
sep_algo.mu_B = mu_B

# Assuming no Bs in the population
trans_prob = np.empty((sep_algo.N+1, 6))

for i in range(sep_algo.N+1):
    trans_prob[i, 0] = sep_algo._prob_A_to_B(i, sep_algo.N - i, 0)
    trans_prob[i, 1] = sep_algo._prob_A_to_WT(i, sep_algo.N - i, 0)
    trans_prob[i, 2] = sep_algo._prob_B_to_A(i, sep_algo.N - i, 0)
    trans_prob[i, 3] = sep_algo._prob_B_to_WT(i, sep_algo.N - i, 0)
    trans_prob[i, 4] = sep_algo._prob_WT_to_A(i, sep_algo.N - i, 0)
    trans_prob[i, 5] = sep_algo._prob_WT_to_B(i, sep_algo.N - i, 0)

In [13]:
s

999

In [8]:
# Trace names - represent the transition probabilities used for the simulation
trace_name = ['A->B', 'A->WT', 'B->A', 'B->WT', 'WT->A', 'WT->B']

fig = go.Figure()

# Add traces of the transition probabilities
for c in range(trans_prob.shape[1]):
    fig.add_trace(
        go.Scatter(
            y=trans_prob[:, c],
            x=list(range(sep_algo.N+1)),
            mode='lines',
            name=trace_name[c],
            line_color=colours[c]
        )
    )

fig.update_layout(
    title='Transition probabilities for edge case with no B cells',
    width=1000, 
    height=600,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
    )